In [ ]:
from datetime import datetime
import itertools
import json
import math
import os
import sys

import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import scipy
from scipy.spatial import KDTree
import seaborn as sns
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

from utils import data_utils, data_loader, shape_utils

importlib.reload(data_utils)
importlib.reload(data_loader)
importlib.reload(shape_utils)

from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

RUN_FOLDER = "../results/debug/"

In [80]:
kcm_inputs = data_utils.load_all_inputs(RUN_FOLDER, "kcm/")
atb_inputs = data_utils.load_all_inputs(RUN_FOLDER, "atb/")

In [ ]:
# Sample shingle ids and choose dist/time ranges to test
shingle_ids = pd.unique(kcm_inputs['test_traces']['shingle_id'])
shingle_id_sample = np.random.choice(shingle_ids, 100)
d_buffers = np.linspace(1,10000,20)
t_buffers = np.linspace(1,60*60*6,20)

# Collect metrics for each combination of d/t
target = np.empty((len(shingle_id_sample)))
adj = np.empty((len(shingle_id_sample), len(d_buffers), len(t_buffers)))
for i, id in enumerate(shingle_id_sample):
    print(i)
    for d, dist in enumerate(d_buffers):
        for t, time in enumerate(t_buffers):
            adj_pts = None
            # Keep record of feature for each combination of dist and time buffers
            shingle_pts, adj_pts = shape_utils.get_adjacent_points(kcm_inputs['test_traces'], id, dist, time)
            if adj_pts is None:
                adj[i,d,t] = np.nan
            else:
                adj[i,d,t] = np.mean(adj_pts.speed_m_s)
    # Shingle data is same regardless of t_buffer and d_buffer
    target[i] = np.mean(shingle_pts.speed_m_s)
# Fill cases with no adjacent points with the average
adj[np.isnan(adj)] = np.mean(adj[~np.isnan(adj)])

In [ ]:
# Rows, Columns
r_values = np.empty((adj.shape[1], adj.shape[2]))
for d in range(adj.shape[1]):
    for t in range(adj.shape[2]):
        R2, x_plt, y_plt = shape_utils.fit_poly(adj[:,d,t], target)
        r_values[d,t] = R2
r_values

In [ ]:
fig = px.imshow(r_values, labels={"x":"Time","y":"Dist"})
fig.show()

In [ ]:
# zero_idx_adj = adj_metric!=0.0
# zero_idx_target = target_metric!=0.0
# x = np.log(adj_metric[zero_idx_adj & zero_idx_target])
# y = np.log(target_metric[zero_idx_adj & zero_idx_target])